# 项目3.1-神经网络压缩（模型架构设计）

## 友情提示
同学们可以前往课程作业区先行动手尝试 ！！！

## 项目描述

本次作业为设计一个小网络(student net)架构，为之后的其他模型压缩作业做准备。
         
如果需要

## 数据集介绍

本作业使用的数据集为food-11数据集，共有11类“
其中每一类对应的标签为：0-面包，1-乳制品，2-甜点，3-鸡蛋，4-油炸食品，5-肉类，6-面条/意大利面，7-米饭，8-海鲜，9-汤，10-蔬菜/水果
训练集: 9866张
验证集: 3430张
测试集: 3347张

**数据格式**

下载 zip 档后解压缩会有三个资料夹，分别为training（训练集）、validation（验证集合）以及 testing（测试集）
training 以及 validation 中的照片名称格式为 [类别]_[编号].jpg，例如 3_100.jpg 即为类别 3 的照片（编号不重要）

## 项目要求
网络架构设计: 将原始的layer用更少的参数来表现。(例如 Convolution ￫ Depthwise & Pointwise Convolution)

## 数据准备

In [1]:
!mkdir data/food-11 
!unzip -d data/food-11 data/data58106/food-11.zip

## 环境配置/安装

无

## 简介

本项目任务是模型压缩 - Neural Network Compression。

Compression有很多种门派，在这裡我们会介绍上课出现过的其中四种，分别是:

* 知识蒸馏 Knowledge Distillation
* 网路剪枝 Network Pruning
* 用少量参数来做CNN Architecture Design
* 参数量化 Weight Quantization       

在这个notebook中我们会介绍MobileNet v1的Architecture Design

# 网络架构设计

## 深度可分离卷积     
![](https://ai-studio-static-online.cdn.bcebos.com/f68ad9db59c443528a7f4440dbd7386dec97436e7c524376a2e730b76f07425a)    
> 蓝色为上下层Channel的关係，绿色则为该Receptive Field的扩张。     
> (图片引用自arxiv:1810.04231)     

(a) 就是一般的卷积神经网络，所以他的Weight连接方式会跟Fully Connected一样，只差在原本在FC是用数字相乘后相加，Convolution Layer是图片卷积后相加。     

(b) DW(Depthwise Convolution Layer)你可以想像成一张feature map各自过**一个filter**处理后，再用PW(Pointwise Convolution Layer)把所有feature map的单个pixel资讯合在一起(就是1个pixel的Fully Connected Layer)。     

(c) GC(Group Convolution Layer)就是把feature map分组，让他们自己过Convolution Layer后再重新Concat起来。算是一般的Convolution和Depthwise Convolution的折衷版。**所以说，Group Convolution的Group=Input Feautures数就会是Depthwise Convolution(因为每个Channel都各自独立)，Group=1就会是一般的Convolution(因为就等于没有Group)。**     

![](https://ai-studio-static-online.cdn.bcebos.com/fb20e573dcbc49d1921a8bdf12a8e53fa447ec879c43459fbdc9ea4c3c4f393c)         
       
## 细节     
```python       
# 一般的Convolution, weight大小 = in_chs * out_chs * kernel_size^2       
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding)       
       
# Group Convolution, Group数目可以自行控制，表示要分成几群。其中in_chs和out_chs必须要可以被groups整除。(不然没办法分群。)       
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding, groups=groups)
         
# Depthwise Convolution, 输入chs=输出chs=Groups数目, weight大小 = in_chs * kernel_size^2      
nn.Conv2d(in_chs, out_chs=in_chs, kernel_size, stride, padding, groups=in_chs)      
       
# Pointwise Convolution, 也就是1 by 1 convolution, weight大小 = in_chs * out_chs       
nn.Conv2d(in_chs, out_chs, 1)       
```

# 模型

* 训练的部分请参考Network Pruning、Knowledge Distillation，或直接只用Hw3即可。

> 注记: 这边把各个Block多用一层Sequential包起来是因为Network Pruning的时候抓Layer比较方便。

In [3]:
import paddle.nn as nn
import paddle


class StudentNet(paddle.nn.Layer):
    '''
      在这个网络里，我们会使用Depthwise & Pointwise Convolution Layer来做model。
      你会发现，将原本的Convolution Layer换成Dw & Pw后，Accuracy通常不会降很多。

      另外，取名为StudentNet是因为这个Model等会要做Knowledge Distillation。
    '''

    def __init__(self, base=16, width_mult=1):
        '''
          参数:
            base: 这个model一开始的通道数量，每过一层都会*2，直到base*16为止。
            width_mult: 为了之后的Network Pruning使用，在base*8 chs的Layer上会 * width_mult代表剪枝后的通道数量。
        '''
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]

        bandwidth = [base * m for m in multiplier]

        for i in range(3, 7):
            bandwidth[i] = int(bandwidth[i] * width_mult)

        self.cnn = nn.Sequential(
            nn.Sequential(

                nn.Conv2D(3, bandwidth[0], 3, 1, 1),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.MaxPool2D(2, 2, 0),
            ),
            nn.Sequential(
                
                nn.Conv2D(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                nn.BatchNorm2D(bandwidth[0]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[0], bandwidth[1], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2D(bandwidth[1]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2D(bandwidth[2]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2D(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2D(bandwidth[3]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[3], bandwidth[4], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2D(bandwidth[4]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[4], bandwidth[5], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2D(bandwidth[5]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[5], bandwidth[6], 1),
            ),

            nn.Sequential(
                nn.Conv2D(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2D(bandwidth[6]),
                nn.ReLU6(),
                nn.Conv2D(bandwidth[6], bandwidth[7], 1),
            ),

            nn.AdaptiveAvgPool2D((1, 1)),
        )
        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 11),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.reshape((out.shape[0], -1))
        return self.fc(out)